In [1]:
import altair as alt
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/fund_split/"

In [2]:
# move filter location to the top

display(HTML(        """<style>form.vega-bindings {  position: absolute;  right: 0px;  top: 0px;}</style>"""))

# SB125 Fund Split Analysis

## Question:
- How did RTAs split SB125 funds between operations and capital?

In [3]:
all_melt = pd.read_parquet(f"{GCS_PATH}all_fund_requests_melt.parquet")

In [4]:
by_source = (
    all_melt.groupby(["fund source"])
    .agg({"fund amount": "sum", "rtpa": "nunique"})
    .reset_index()
)
by_type = (
    all_melt.groupby(["project type"])
    .agg(
        {
            "fund amount": "sum",
        }
    )
    .reset_index()
)
by_year = (
    all_melt.groupby(["fiscal year", "project type"])
    .agg({"fund amount": "sum"})
    .reset_index()
)
by_rtpa = (
    all_melt.groupby(["rtpa", "project type", "fiscal year"])
    .agg({"fund amount": "sum"})
    .reset_index()
)

by_agency = (
    all_melt.groupby(["implementing agenc-y/-ies", "project type", "fiscal year"])
    .agg({"fund amount":"sum"})
).reset_index()

# aggregate by rtpa, agency, project type and FY
rtpa_group = (
    all_melt.groupby(
        ["rtpa", "implementing agenc-y/-ies", "project type", "fiscal year"]
    )
    .agg({"fund amount": "sum"})
    .reset_index()
)

In [5]:
stack_chart = (
    alt.Chart(by_year)
    .mark_bar(point=True)
    .encode(
        y="fiscal year",
        x="fund amount",
        color="project type",
        tooltip=["project type", "fund amount"],
    )
    .properties(
        title="SB125 allocation funds by capital/operating expenses, by fiscal year",
        width=600,
        height=300,
    )
)

stack_text = stack_chart.mark_text(
    align="left",
    dx=-10,
    dy=10
).encode(
    text="fund amount", color=alt.value("black")
)

In [6]:
overall_chart = (
    alt.Chart(by_type)
    .mark_bar(point=True)
    .encode(
        y="project type",
        x="fund amount",
        color="project type",
        tooltip=["project type", "fund amount"],
    )
    .properties(
        title="Overall split of SB125 allocation funds by capital/operating expenses",
        width=600,
        height=300,
    )
)

overall_text = overall_chart.mark_text(
    align="left",
    dx=0,
    dy=10
).encode(
    text="fund amount", color=alt.value("black")
)

In [7]:
source_chart = (
    alt.Chart(by_source)
    .mark_bar(point=True)
    .encode(
        y="fund source",
        x="fund amount",
        # color="project type",
        tooltip=["fund source", "fund amount"],
    )
    .properties(title="Allocation funds by funding source", width=600, height=300)
)

source_text = source_chart.mark_text(
    align="left",
    dx=0,
    dy=10
).encode(
    text="fund amount", color=alt.value("black")
)

In [8]:
#initial bar chart
rtpa_bar_chart_2 = alt.Chart(rtpa_group).mark_line(point=True).encode(
    alt.Y("fund amount"),
    alt.X("fiscal year"),
    color="project type",
    #tooltip=["rtpa","implementing agenc-y/-ies","project type", "fund amount", "fiscal year"]
)

# create drop down

## list of rtpas
rtpa_list = list(rtpa_group["rtpa"].unique())

## actual drop down mechanism
rtpa_dropdown = alt.binding_select(
    options = rtpa_list,
    name= "Select RTPA"
)

rtpa_selector = alt.selection_point(fields=['rtpa'], value="VCTC", bind=rtpa_dropdown)

# data label text
rtpa_chart_text_2 = rtpa_bar_chart_2.mark_text(
    align="center",
    dx=0,
    dy=-10,
    
    ).encode(text="fund amount:N", color=alt.value("black"))

# combine bar and text charts
rtpa_chart_combo_2 = (rtpa_bar_chart_2 + rtpa_chart_text_2).properties(
    title="Implementing agency funding split",
    width=300,
    height=150
)

# add facets by agency
rtpa_facet_2 = rtpa_chart_combo_2.add_params(
    rtpa_selector
).facet(
    facet=alt.Facet("implementing agenc-y/-ies", title="SB 125 Funding by capital/operating cost by fiscal year, by agencies in RTPAs"), columns=3
).resolve_scale(
    x="independent", y="independent"
).transform_filter(
    rtpa_selector
)



In [9]:
# list of RTPAs
rtpa_list = list(rtpa_group["rtpa"].unique())

#selectors
rtpa_dropdown = alt.binding_select(
    options = rtpa_list,
    name= "Select RTPA"
)

rtpa_selector = alt.selection_point(
    fields=['rtpa'], value="VCTC", bind=rtpa_dropdown
)

#opx chart
opx_chart = alt.Chart(rtpa_group[rtpa_group['project type']=="operating"], title="Operating Funds").mark_bar().encode(
    alt.Y("implementing agenc-y/-ies").title("Agency"),
    alt.X("fund amount"),
    alt.YOffset("fiscal year"),
    alt.Color("implementing agenc-y/-ies"),
    tooltip=["fund amount", "fiscal year"]
).add_params(
    rtpa_selector
).transform_filter(
    rtpa_selector
).properties(
        width=400,
        height=400,
).resolve_scale(
    x="shared", y="independent"
)

opx_text = opx_chart.mark_text(
    align="left",
    dx=0,
    dy=0
).encode(
    text="fund amount", color=alt.value("black")
)


opx_chart_text = (opx_chart + opx_text)


#capx chart
capx_chart = opx_chart.properties(
    data=rtpa_group[rtpa_group['project type']=="capital"], title="Capital Funds"
).transform_filter(
    rtpa_selector
)

capx_text = capx_chart.mark_text(
    align="left",
    dx=0,
    dy=0
).encode(
    text="fund amount", color=alt.value("black")
)

capx_chart_text = (capx_chart + capx_text)

opx_capx_chart = alt.hconcat(
    opx_chart, capx_chart
).resolve_scale(
    x="shared", y="independent"
).properties(
    title=alt.TitleParams(
        text="RTPA split of Operating and Capital Funds Requested by Agencies",
        subtitle="Separated by Fiscal Year"
    )
)




In [10]:
display(
    overall_chart + overall_text,
    stack_chart + stack_text,
    source_chart + source_text,
)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [11]:
# render chart
rtpa_facet_2

alt.FacetChart(...)

In [12]:
opx_capx_chart

alt.HConcatChart(...)